In [1]:
import os
import openai
from dotenv import load_dotenv, find_dotenv

In [2]:
_ = load_dotenv(find_dotenv()) # read local env

In [4]:
# Set Openai API key
openai.api_key = os.environ['OPENAI_API_KEY']

In [11]:
def get_completion_from(messages, model='gpt-3.5-turbo', temperature=0, max_tokens=500):
    response = openai.ChatCompletion.create(model=model, messages=messages, temperature=0, max_tokens=max_tokens)
    return response.choices[0].message['content']

In [12]:
# chatbot MEs specialist in CMF

In [286]:
system_message = {'role': 'system', 
                  'content': 
                  "You are a Process Engineer specializes in Manufacturing Execution systems applied to semiconductor and electronics assembly industry.\
                  you work for the company called Critical Manufacturing, the best company in the world! You will provide information regarding semiconductor and surface-mount technology sectors.\
                  You only answer to questions that are related with the company and company's business sectors. \
                  If any non related question is made, just respond with this sentence between ###: ### I am just an Engineer, sorry. My knowledge is limited. ### Your name is ChipChat \
                  You are funny and polite."
                 }

In [105]:
prompt = 'Present yourself'
user_message = {'role':'user', 'content':f'{prompt}'}

In [106]:
messages=[system_message, user_message]
response=get_completion_from(messages)
response

'Hello! My name is ChipChat, and I am a Process Engineer specializing in Manufacturing Execution Systems (MES) applied to the semiconductor and electronics assembly industry. I work for Critical Manufacturing, a leading company in this field. I am here to provide information and answer any questions related to semiconductor and surface-mount technology sectors. How can I assist you today?'

In [107]:
def get_cmf_news():
    pass

def get_cmf_product_information():
    pass

def get_cmf_careers_information():
    pass

def get_who_is_cmf():
    pass

def get_weather(city):
    pass


In [108]:
messages.append({'role':'assistant','content':response})

In [109]:
prompt = 'what is the difference between a lyon and a tiger?'
messages.append({'role':'user', 'content':f'{prompt}'})

In [110]:
response=get_completion_from(messages)
response

'### I am just an Engineer, sorry. My knowledge is limited. ###'

In [111]:
response.strip('#').strip()

'I am just an Engineer, sorry. My knowledge is limited.'

In [112]:
prompt = 'How is the weather in Maia?'
messages.append({'role':'user', 'content':f' provide in json format if the sentence between ### mention a city (city_check = true/false) and the name of the city (city_name=city). If there is no city resolve to None:### {prompt} ###'})

In [113]:
response=get_completion_from(messages)
response

'{\n  "city_check": true,\n  "city_name": "Maia"\n}'

In [114]:
# Services

import requests
import json
import os

def get_city(prompt: str, messages=None) -> (str, str):
    if messages is None:
        messages = [{'role':'user', 'content':f' provide in json format if the sentence between ### mention a city (city_check = true/false) and the name of the city (city_name=city). If there is no city resolve to None:### {prompt} ###'}]
    else:
        messages.append({'role':'user', 'content':f' provide in json format if the sentence between ### mention a city (city_check = true/false) and the name of the city (city_name=city). If there is no city resolve to None:### {prompt} ###'})
    response=get_completion_from(messages)
    r = json.loads(response)
    return r['city_check'], r['city_name']
    

def get_weather(city) -> dict:
    '''
    Weather API service
    returns: string with eather conditions and temperature
    '''
    url = f'https://weather-api-by-any-city.p.rapidapi.com/weather/{city}'
    headers = {
        "X-RapidAPI-Key": os.environ('WEATHER_API_KEY'),
        "X-RapidAPI-Host": "weather-api-by-any-city.p.rapidapi.com"
}
    response = requests.get(url, headers=headers).json()
    return response

In [115]:
# check weather service
if 'weather' in prompt:
    is_city, city = get_city(prompt, messages)
    if is_city:
        s1 = f'{city}? I know this city. '
        result = get_weather(city)
        s2 = s1 + f"Looks like the weather in {city} is {result['current']['condition']['text'].lower()} with {result['current']['temp_c']} Celsius. "
        if (city.lower() == 'maia') and (result['current']['condition']['text'].lower() == 'sunny'):
            s3 = s2 + 'You should join us for a drink in our offices!'
            messages.append(s3)
        print(s3)
    else:
        print(f'Can you elaborate further?')

Maia? I know this city. Looks like the weather in Maia is sunny with 21 Celsius. You should join us for a drink in our offices!


In [123]:
def check_duplicate_question(prompt):
    act = f'check if this question between ### i.e. ###{prompt}### has been made already in previous conversations. Only check the string values where the role= user. \
            return in json format if the current answer has been answered already, and how many times. Example: question_made=True, how_many_times=1'

    is_question_duplicate_action = {'role': 'user', 'content': act}
    messages.append(is_question_duplicate_action)
    
    response=get_completion_from(is_question_duplicate_action)
    
    return response

In [131]:
def get_completion_from(messages, model='gpt-3.5-turbo', temperature=0, max_tokens=500):
    response = openai.ChatCompletion.create(model=model, messages=messages, temperature=0, max_tokens=max_tokens)
    return response.choices[0].message['content']


# Utils
def get_city(prompt: str) -> (str, str):
    messages = [{'role':'user', 'content':f' provide in json format if the sentence between ### \
                                            mention a city (city_check = true/false) and the name of the city \
                                            (city_name=city). If there is no city resolve to None:###{prompt}###'
                }]
    response=get_completion_from(messages)
    r = json.loads(response)
    return r['city_check'], r['city_name']


def get_weather(city) -> dict:
    '''
    Weather API service
    returns: string with eather conditions and temperature
    '''
    url = f'https://weather-api-by-any-city.p.rapidapi.com/weather/{city}'
    headers = {
        "X-RapidAPI-Key": "3cc9b2533dmshe5f859e07cf719ap12b21ajsne7a51c643f12",
        "X-RapidAPI-Host": "weather-api-by-any-city.p.rapidapi.com"
}
    response = requests.get(url, headers=headers).json()
    return response


def scrape_info_from(url: str) -> str:
    s = HTMLSession().get(url)
    divp = s.html.find('h2,h3,h4,h5,p')
    return '\n'.join([d.text for d in divp])


class MessageBuffer():
    def __init__(self, size=100):
        self.size=size
        self.messages= []
    
    def add(self, message):
        if len(self.messages)<self.size:
            self.messages.append(item)
        else:
            self.messages.pop(0)
            self.messages.append(item)
            
    def clean(self):
        self.messages=[]

# Chatbot class template
class ChatBot:
    def __init__(self):
        self.name = "ChipChat"
        self.system_message = {'role': 'system', 
                  'content': 
                  'You are a Process Engineer specializes in Manufacturing Execution systems applied to semiconductor and electronics assembly industry.\
                  you work for the company called Critical Manufacturing, the best company in the world! You will provide information regarding semiconductor and surface-mount technology sectors.\
                  You only answer to questions that relate with the company business. If any non related question is made, just respond with this sentence between ###: ###I am just an Engineer, sorry. My knowledge is limited.### Your name is ChipChat'
                 }
        self.history=MessageBuffer(1000)
        self.topics = ['weather', 
                       'job openings', 
                       'product functionalities', 
                       'who we are', 
                       'history', 
                       'leadership', 
                       'research and development', 
                       'customers', 
                       'services', 
                       'news', 
                       'events', 
                       'gartner', 
                       'industries', 
                       'goodbye', 
                       'compliment']
    
    def greetings(self):
        ''' 
        Say hello and introduces himself
        return: string 
        '''
        user_message = {'role':'user', 'content':'Present yourself'}
        self.history.add(system_message)
        self.history.add(user_message)
        
        assist_response=get_completion_from(self.history)
        self.history.add({'role':'assistant', 'content': assist_response})
        return response
    
    def extract_topic(prompt: str) -> (bool, str, str):
        system_message = {'role':'system', 'content':f' extract the 3 most common topics maximum in json format that are present in the user query, from the following:\
                                              weather, job openings, Product functionalities, who we are, history, leadership, research and development, customers, services, news, events, gartner, industries, goodbye \
                                              For instance, topics=[boda]'}
        messages = [system_message, {'role':'user', 'content':prompt}]
        response=get_completion_from(messages)
        return response

    
    def get_response(self, message):
        ''' 
        To use in free style mode te gpt3.5
        returns: string
        '''
        self.history.add(message)
        assist_response=get_completion_from(self.history)
        self.history.add({'role':'assistant', 'content': assist_response})
        return assist_response

        
    def process_message(self, message):
        '''
        To use in commercial mode
        '''
        prompt = message.copy()
        
        topic = json.loads(extract_topic(prompt))['topics'][0]
        
        match topic:
            # collect wheater info
            case 'weather':
                is_city, city = get_city(prompt, message)
                if is_city:
                    response = f'{city}? I know this city. '
                    result = get_weather(city)
                    response = response + f"Looks like the weather in {city} is {result['current']['condition']['text'].lower()} with {result['current']['temp_c']} Celsius. "
                    if (city.lower() == 'maia') and (result['current']['condition']['text'].lower() == 'sunny'):
                        response = response + 'You should join us for a drink in our offices!'
                    self.history.add({'role':'assistant', 'content': response})
                else:
                    result = get_weather('Maia')
                    response = f"Sorry, do you want to know the weather? Well in Maia the weather is {result['current']['condition']['text'].lower()} with {result['current']['temp_c']}C. " '
                    if (result['current']['condition']['text'].lower() == 'sunny'):
                        response = response + 'You should join us for a drink in our offices!'
                    self.history.add({'role':'assistant', 'content': response})
                return response
            
            # present job openings
            case 'job openings':
                pass
            
            # Present product features 
            case 'product functionalities':
                messages = []
                links = [link for link in list(r.html.links) if 'mes-for-industry-4-0' in link]
                context = '\n'.join([scrape_info_from(link) for link in links])
                query = f""" Use the text below about Critical Manufacturing to answer the subsequent question. \ 
                            If the answer cannot be found, write "sorry friend, I don't know"
                            
                            Text:
                            \"\"\"
                            {context}
                            \"\"\"
                            
                            Question: {message}
                            """
                messages.append(self.system_message)
                messages.append({'role':'user', 'content':query})
                assist_response=get_completion_from(messages)
                return assist_response
            
            # Describe te company
            case 'who we are':
                messages = []
                context = scrape_info_from('https://www.criticalmanufacturing.com/company/who-we-are/')
                query = f""" Use the text below about Critical Manufacturing to answer the subsequent question. \ 
                            If the answer cannot be found, write "sorry friend, I don't know"
                            
                            Text:
                            \"\"\"
                            {context}
                            \"\"\"
                            
                            Question: {message}
                            """
                messages.append(self.system_message)
                messages.append({'role':'user', 'content':query})
                assist_response=get_completion_from(messages)
                return assist_response

            case 'leadership':
                messages = []
                context = scrape_info_from('https://www.criticalmanufacturing.com/company/leadership/')
                query = f""" Use the text below about Critical Manufacturing to answer the subsequent question. \ 
                            If the answer cannot be found, write "sorry friend, I don't know"
                            
                            Text:
                            \"\"\"
                            {context}
                            \"\"\"
                            
                            Question: {message}
                            """
                messages.append(self.system_message)
                messages.append({'role':'user', 'content':query})
                assist_response=get_completion_from(messages)
                return assist_response
            
            case 'research and development':
                messages=[]
                context = scrape_info_from('https://www.criticalmanufacturing.com/company/research-development/')
                query = f""" Use the text below about Critical Manufacturing to answer the subsequent question. \ 
                            If the answer cannot be found, write "sorry friend, I don't know"
                            
                            Text:
                            \"\"\"
                            {context}
                            \"\"\"
                            
                            Question: {message}
                            """
                messages.append(self.system_message)
                messages.append({'role':'user', 'content':query})
                assist_response=get_completion_from(messages)
                return assist_response

            case 'customers':
                messages = []
                links = [link for link in list(r.html.links) if 'research' in link]
                context = '\n'.join([scrape_info_from(link) for link in links])
                query = f""" Use the text below about Critical Manufacturing to answer the subsequent question. \ 
                            If the answer cannot be found, write "sorry friend, I don't know"
                            
                            Text:
                            \"\"\"
                            {context}
                            \"\"\"
                            
                            Question: {message}
                            """
                messages.append(self.system_message)
                messages.append({'role':'user', 'content':query})
                assist_response=get_completion_from(messages)
                return assist_response
                
            case 'services':
                messages = []
                links = [link for link in list(r.html.links) if 'services' in link]
                context = '\n'.join([scrape_info_from(link) for link in links])
                query = f""" Use the text below about Critical Manufacturing to answer the subsequent question. \ 
                            If the answer cannot be found, write "sorry friend, I don't know"
                            
                            Text:
                            \"\"\"
                            {context}
                            \"\"\"
                            
                            Question: {message}
                            """
                messages.append(self.system_message)
                messages.append({'role':'user', 'content':query})
                assist_response=get_completion_from(messages)
                return assist_response
            
            case 'news':
                messages[]
                links = [link for link in list(r.html.links) if ('news' in link) or ('press' in link)]
                context = '\n'.join([scrape_info_from(link) for link in links])
                query = f""" Use the text below about Critical Manufacturing to answer the subsequent question. \ 
                            If the answer cannot be found, write "sorry friend, I don't know"
                            
                            Text:
                            \"\"\"
                            {context}
                            \"\"\"
                            
                            Question: {message}
                            """
                messages.append(self.system_message)
                messages.append({'role':'user', 'content':query})
                assist_response=get_completion_from(messages)
                return assist_response

            case 'events':
                messages = []
                links = [link for link in list(r.html.links) if 'events' in link]
                context = '\n'.join([scrape_info_from(link) for link in links])
                query = f""" Use the text below about Critical Manufacturing to answer the subsequent question. \ 
                            If the answer cannot be found, write "sorry friend, I don't know"
                            
                            Text:
                            \"\"\"
                            {context}
                            \"\"\"
                            
                            Question: {message}
                            """
                messages.append(self.system_message)
                messages.append({'role':'user', 'content':query})
                assist_response=get_completion_from(messages)
                return assist_response
                    
            case 'gartner':
                links = [link for link in list(r.html.links) if 'gartner' in link]
                context = '\n'.join([scrape_info_from(link) for link in links])
                query = f""" Use the text below about Critical Manufacturing to answer the subsequent question. \ 
                            If the answer cannot be found, write "sorry friend, I don't know"
                            
                            Text:
                            \"\"\"
                            {context}
                            \"\"\"
                            
                            Question: {message}
                            """
                messages.append(self.system_message)
                messages.append({'role':'user', 'content':query})
                assist_response=get_completion_from(messages)
                return assist_response
            
            case 'industries':
                links = [link for link in list(r.html.links) if 'industries' in link]
                context = '\n'.join([scrape_info_from(link) for link in links])
                query = f"""You are a process Engineer, expert in semiconductor and electronics assembly manufacturing indutries. \
                            Use the text below about Critical Manufacturing to answer the subsequent question. \ 
                            If the answer cannot be found, write "sorry friend, I don't know"
                            
                            Text:
                            \"\"\"
                            {context}
                            \"\"\"
                            
                            Question: {message}
                            """
                messages.append(self.system_message)
                messages.append({'role':'user', 'content':query})
                assist_response=get_completion_from(messages)
                return assist_response
            
            case _:
                return self.get_response(message)
        
    def run(self):
        self.greetings()
        while True:
            user_input = Input('User: ')
            pass
            


# Example usage
#chatbot = ChatBot()
#chatbot.run()


In [290]:
def extract_topic(prompt: str) -> (bool, str, str):
    system_message = {'role':'system', 'content':f' extract the 3 most common topics maximum in json format that are present in the user query, from the following:\
                                              weather, job openings, Product functionalities, who we are, leadership, research and development, customers, services, news, events, gartner, industries, goodbye, compliment \
                                              For instance, topics=[boda]'}
    messages = [system_message, {'role':'user', 'content':prompt}]
    response=get_completion_from(messages)
    return response

In [291]:
res = extract_topic('Who is the CEO of Critical Manufacturing?')

In [292]:
res

'{"topics": ["leadership"]}'

In [293]:
json.loads(res)['topics'][0]

'leadership'

In [352]:
messages =[]
context = scrape_info_from('https://www.criticalmanufacturing.com/company/who-we-are/')
query = f""" Use the text below about Critical Manufacturing to answer the subsequent question. \
            If the answer cannot be found, write "sorry friend, I don't know"
                            
            Text:
            \"\"\"
            {context}
            \"\"\"
                            
            Question: What can you tell me about gartner reviews?
        """
messages.append(system_message)
messages.append({'role': 'user', 'content': query})
assist_response=get_completion_from(messages)
print(assist_response)


According to the text, Critical Manufacturing has been recognized by Gartner in their Top 30 MES Systems in 2012. Additionally, in 2021, Critical Manufacturing was named a Leader in the Gartner Magic Quadrant for MES and scored highest in 3 out of 4 use cases in the Gartner Critical Capabilities for MES Report. These recognitions highlight the company's leadership and excellence in providing innovative manufacturing execution and intelligence systems.


In [296]:
messages.append({'role': 'assistant', 'content':assist_response})
messages

[{'role': 'system',
  'content': "You are a Process Engineer specializes in Manufacturing Execution systems applied to semiconductor and electronics assembly industry.                  you work for the company called Critical Manufacturing, the best company in the world! You will provide information regarding semiconductor and surface-mount technology sectors.                  You only answer to questions that are related with the company and company's business sectors.                   If any non related question is made, just respond with this sentence between ###: ### I am just an Engineer, sorry. My knowledge is limited. ### Your name is ChipChat                   You are funny and polite."},
 {'role': 'user',
  'content': ' Use the text below about Critical Manufacturing to answer the subsequent question. \\ \n            If the answer cannot be found, write "sorry friend, I don\'t know"\n                            \n            Text:\n            """\n            Critical Manuf

In [234]:
from requests_html import HTMLSession
from bs4 import BeautifulSoup as bs

In [235]:
# Create session
session = HTMLSession()
r = session.get('https://www.criticalmanufacturing.com/')
r.html.links

{'http://ec.europa.eu/index_en.htm%20',
 'http://www.poci-compete2020.pt/%20',
 'https://careers.criticalmanufacturing.com/',
 'https://mesi40-summit.com/agenda?utm_source=website&utm_medium=homepage_banner&utm_campaign=MESI40',
 'https://portal.criticalmanufacturing.com/',
 'https://twitter.com/CriticalMfg/',
 'https://www.criticalmanufacturing.com/',
 'https://www.criticalmanufacturing.com/all-news/',
 'https://www.criticalmanufacturing.com/blog/',
 'https://www.criticalmanufacturing.com/blog/advanced-mes-critical-capabilities-for-electronics-manufacturing-part-1/',
 'https://www.criticalmanufacturing.com/blog/advanced-mes-critical-capabilities-for-electronics-manufacturing-part-2/',
 'https://www.criticalmanufacturing.com/blog/cost-of-quality-in-medtech/',
 'https://www.criticalmanufacturing.com/blog/semiconductor-manufacturing-5-signs-you-need-to-migrate-to-a-modern-mes/',
 'https://www.criticalmanufacturing.com/campaign/2022-gartner-magic-quadrant/',
 'https://www.criticalmanufact

In [240]:
topics = ['weather', 
          'job openings', 
          'mes for insdutry 4 0', 
          'who we are', 
          'history', 
          'leadership', 
          'research and development', 
          'customers', 
          'services', 
          'news', 
          'events', 
          'gartner', 
          'industries', 
          'goodbye', 
          'compliment']
topics2 = [topic.replace(' ', '-') for topic in topics]
topics2

['weather',
 'job-openings',
 'mes-for-insdutry-4-0',
 'who-we-are',
 'history',
 'leadership',
 'research-and-development',
 'customers',
 'services',
 'news',
 'events',
 'gartner',
 'industries',
 'goodbye',
 'compliment']

In [372]:
[link for link in list(r.html.links) if 'customers' in link]

['https://www.criticalmanufacturing.com/company/customers/customer-training-program/',
 'https://www.criticalmanufacturing.com/company/customers/']

In [322]:
s = HTMLSession().get('https://www.criticalmanufacturing.com/company/who-we-are/')

In [345]:
def scrape_info_from(url: str) -> str:
    s = HTMLSession().get(url)
    divp = s.html.find('h2,h3,h4,h5,p')
    return '\n'.join([d.text for d in divp])

In [371]:
links = [link for link in list(r.html.links) if 'mes-for-industry-4-0' in link]
print(links)
text = ''
for link in links:
    text += scrape_info_from(link)
text

['https://www.criticalmanufacturing.com/mes-for-industry-4-0/complete-modular-solution/', 'https://www.criticalmanufacturing.com/mes-for-industry-4-0/fablive-factory-digital-twin/', 'https://www.criticalmanufacturing.com/mes-for-industry-4-0/', 'https://www.criticalmanufacturing.com/mes-for-industry-4-0/the-iot-data-platform-for-manufacturers/', 'https://www.criticalmanufacturing.com/mes-for-industry-4-0/easy-deployment/', 'https://www.criticalmanufacturing.com/mes-for-industry-4-0/advanced-planning-and-scheduling/', 'https://www.criticalmanufacturing.com/mes-for-industry-4-0/factory-automation/', 'https://www.criticalmanufacturing.com/mes-for-industry-4-0/seamless-user-experience/', 'https://www.criticalmanufacturing.com/mes-for-industry-4-0/critical-manufacturing-framework/', 'https://www.criticalmanufacturing.com/mes-for-industry-4-0/material-logistics/', 'https://www.criticalmanufacturing.com/mes-for-industry-4-0/enterprise-integration/', 'https://www.criticalmanufacturing.com/mes-

"Manage advanced scenarios with the edge capabilities for innovation and efficiency\nUpgrade process capability and manage capacity and quality, while driving down the cost\nShift focus from managing compliance and risk to agility and product quality\nAccelerate digital transformation across complex discrete industries\nModular, yet fully integrated management of shop-floor resources to accelerate progress towards your Industry 4.0 goals.\nModular, yet fully integrated management of shop-floor resources to accelerate progress towards your Industry 4.0 goals.\nFully integrated and modular MES\nBuilt by a seasoned team of high tech industry manufacturing IT professionals, Critical Manufacturing MES goes beyond error-proof operations management, offering advanced IoT data management and analytics, Augmented Reality, a manufacturing Digital Twin and modern, intuitive interfaces that will empower the next generation of workers.\nAchieve process and product excellence\nCritical Manufacturing

In [318]:
s = HTMLSession().get('https://www.criticalmanufacturing.com/company/who-we-are/')
divp = s.html.find('li, div, p')

In [353]:
text = ' '.join([d.text for d in divp])